<span style="font-size: 15pt">Access This Presentation Online:</span>
<h1 style="margin: 0; font-size: 20pt">https://webgeeks.netlify.com/fullstack.html</h1>

<h1 style="font-size: 30pt"><em>Full Stack Web Dev w/ EmberJS and Django</em></h1><h3 style="display: inline;">by Michael Johnson</h3>
<div>
<img src="https://emberjs.com/images/tomster-sm-45d6007ea34621fb603f29bd6a77798a.png" alt="Ember Tomster Logo" width="100" style="overflow: hidden; display: inline; margin-left: 50px; margin-right: 50px;"><strong style="font-size: 40pt">+</strong><img src="https://i0.wp.com/www.skysilk.com/blog/wp-content/uploads/2017/11/python-django-logo.jpg?fit=1280%2C720&ssl=1" alt="Python and Django" width="200" style="overflow: hidden; display: inline; margin-left: 50px; margin-right: 50px;">
</div>

<h1>The Chesse - Django API with  Ember Consumer</h1>
<img src="./static/simple-api-github.png" alt="Python and Django" width="500" style="overflow: hidden; display: inline; margin-left: 50px; margin-right: 50px;"><br>

<span style="font-size: 20pt">Check out full length tutorial:</span>
<h1 style="margin: 0; font-size: 30pt">github.com/micleners/DSM-Web-Geeks</h1>

# Previous Django Rest Framework Talk/Tutorial

Check out the tutorial at:
### github.com/micleners/pyowa-drf-demo
![Pyowa DRF Presentation](./static/pyowa-screenshots.png "Pyowa DRF Presentation")



## Backend Startup

Base folder: `mkdir ember-django && cd ember-django`

Create backend environment with packages: 
- `mkdir events-backend && cd events-backend`
- `pip install pipenv`
- `pipenv install django djangorestframework djangorestframework-jsonapi`

Activate Backend Shell: `pipenv shell`

Start Django Project: `django-admin startproject main_project .`

Create Events App: `python manage.py startapp events`

Start Server: `python manage.py runserver`

Cool - can't really do anything yet tho 😎

### We end up with a file structure like this:

```
▾ events/
  ▸ migrations/
  - __init__.py
  - admin.py
  - apps.py
  - models.py
  - tests.py
  - views.py
▾ main_project/
  - __init__.py
  - settings.py
  - urls.py
  - wsgi.py
- manage.py
- Pipfile
- Pipfile.lock
```

# Housekeeping

### Migrate

Update the DB with the baked in user model:

`python manage.py migrate`

### Inject Apps into `main_project`

```python
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    # import rest_framework for viewset and serializer helpers
    'rest_framework',
    # add `events` application to main project
    'events'
]
```

# COMMIT

Backend django project and events app created

# Overview of Backend


![The parts of our django project](./static/django-architecture.png "The parts of our django project")

## Building Model, Serializer, View and URL for Events

To build an API endpoint (URL to GET, POST, PUT and DELETE) we will do the following:

1. Create our new model (and make migrations)
2. Create a serializer for events
3. Create views for our events
4. Route up our URLs to our views

### Create our Events Model

Add the following code to: `/events-backend/event/models.py`

```python
from django.db import models
from django.utils import timezone

# basic information for our events 
class Event(models.Model):
    # Only required field is title
    title = models.CharField(max_length=256)
    presenter = models.CharField(max_length=256, blank=True)
    # Default time is now if not provided
    time = models.DateTimeField(default=timezone.now)
    location = models.CharField(max_length=256, blank=True)
    description = models.TextField(blank=True)
```

### And migrate the database:

`python manage.py makemigrations`

`python manage.py migrate`

### Make Serializer for Events

Create the file `/events-backend/event/serializers.py` file in events and create serializer:

```python
from django.contrib.auth.models import User, Group
from .models import Event
from rest_framework import serializers


# serializers to convert from Django model objects to JSON
class UserSerializer(serializers.HyperlinkedModelSerializer):
    class Meta:
        model = User
        fields = ('url', 'username', 'email', 'groups')

# or specify all fields
class EventSerializer(serializers.HyperlinkedModelSerializer):
    class Meta:
        model = Event
        fields = '__all__'
```

### Creating a View for our Events

Navigate to the `views.py` file in our events app. Create viewsets:

```python
from django.contrib.auth.models import User
from rest_framework import viewsets
from .models import Event
from .serializers import UserSerializer, EventSerializer


class UserViewSet(viewsets.ModelViewSet):
    """
    API endpoint that allows users to be viewed or edited.
    """
    queryset = User.objects.all().order_by('-date_joined')
    serializer_class = UserSerializer

class EventViewSet(viewsets.ModelViewSet):
    """
    API endpoint that allows groups to be viewed or edited.
    """
    resource_name = 'events'
    queryset = Event.objects.all()
    serializer_class = EventSerializer
```

### Connecting our URLs

In `events-backend/main_project/urls.py`, register the EventViewSet and the UserViewSet

```python
from django.urls import include, path
from django.contrib import admin
from rest_framework import routers
from events import views

router = routers.DefaultRouter()
router.register(r'users', views.UserViewSet)
router.register(r'events', views.EventViewSet)

urlpatterns = [
    path('api/', include(router.urls)),
    path('api-auth/', include('rest_framework.urls', namespace='rest_framework')),
    path('admin/', admin.site.urls),
]
```

## Start Server and Witness the Magic DRF API

Run: `python manage.py runserver`

And navigate to Events API: http://localhost:8000/api/events/

# COMMIT

Backend API endpoint created for events

# What's a serializer doin anyhow?!


![Serialization from Django Model to JSON](./static/drf-serializer-figure.png "Serialization from Django Model to JSON")

# Recap of Backend


![The parts of our django project](./static/django-architecture.png "The parts of our django project")

### What's in this diagram that we have not yet incorporated into our django project?

# Our Data on JSON-API

JSON is awesome - it is very flexible... 🤔 too flexible for it's own good

JSON-API Specification helps us know what to expect!
> By following shared conventions, you can increase productivity, take advantage of generalized tooling, and focus on what matters: your application.

Also, Ember data expects JSON-API. Convenient! No munging Django Models --> Ember Models



![Side by Side comparison of JSON vs JSON API](static/json-vs-jsonapi.png "Side by Side comparison of JSON vs JSON API")

While JSON-API is more verbose, the consisten structure and the information provided is much more useful for microservice communication.

### Activate JSON-API in our Django App

We have already installed DRF JSON-API at the start. Paste the following code in `main_project/settings.py` to set up JSON-API:

```python
# RestFramework settings for DjangoRestFramework-JSONAPI
REST_FRAMEWORK = {
  'PAGE_SIZE': 100,

  'EXCEPTION_HANDLER':
    'rest_framework_json_api.exceptions.exception_handler',

  'DEFAULT_PAGINATION_CLASS':    'rest_framework_json_api.pagination.JsonApiPageNumberPagination',
  'DEFAULT_PARSER_CLASSES': (
    'rest_framework_json_api.parsers.JSONParser',
    'rest_framework.parsers.FormParser',
    'rest_framework.parsers.MultiPartParser'
  ),
  'DEFAULT_RENDERER_CLASSES': (
    'rest_framework_json_api.renderers.JSONRenderer',
    'rest_framework.renderers.BrowsableAPIRenderer',
   ),
   'DEFAULT_METADATA_CLASS': 'rest_framework_json_api.metadata.JSONAPIMetadata',
   'DEFAULT_FILTER_BACKENDS': (
     'rest_framework.filters.OrderingFilter',
    ),
   'ORDERING_PARAM': 'sort',

   'TEST_REQUEST_RENDERER_CLASSES': (
     'rest_framework_json_api.renderers.JSONRenderer',
    ),

   'TEST_REQUEST_DEFAULT_FORMAT': 'vnd.api+json'
}
```

You can refresh the API in your browser and see the data format has changed: http://localhost:8000/api/events - now we can start on our ember app 🥳

# COMMIT

Activate JSON-API at endpoints

# The Cheese


![API Endgoal](./static/simple-api-github.png "API Endgoal")

## Frontend Startup

In base folder: `ember-django`

Install Ember: `npm install -g ember-cli`

Create App: `ember new events-frontend`

Enter into Directory: `cd events-frontend`

Some helpters: 
- `ember install ember-bootstrap`
- `ember install ember-moment`

**Note**: Ember already creates a git repository. You can delete it with `rm -rf .git` if you plan to use a sinple repo in the base folder

## Well Hello, Tomster!
![Tomster Welcome](static/tomster_welcome.png "Ember Welcome Page")

# COMMIT

Started frontend ember application

# Load in data - the classic "GET"

To get our data from the API, we will need to:
1. Set up the API URL in `adapter/application.js`
2. Create a model for our event data
3. Fetch our data in our route javascript file
4. Display the data in the browser

## Set up the API URL in `adapter/application.js`

Create our adapter with Ember CLI: `ember g adapter application`

In the file `app/adapters/application.js` update with:

```javascript
import DS from 'ember-data';
import { computed } from '@ember/object';

export default DS.JSONAPIAdapter.extend({
  host: computed(function(){
    return 'http://localhost:8000';
  }),
  namespace: 'api',
  buildURL: function(type, id, record) {
  //call the default buildURL and then append a slash
  return this._super(type, id, record) + '/';
  }
});
```

Builds URL with: `{ host }` / `{ namespace }` / `{extra bits }` `{append /}` 
     
  --> `http://localhost:8000/` `api/` `events/1` `/`

## Create a model for events

Create an ember model via the CLI to load our API data into:

`ember g model event`

We then go into `create app/models/event.js` and create our model with the same fields as we did in django:

```javascript
import DS from 'ember-data';
import { computed } from '@ember/object'
import moment from 'moment';
const { Model } = DS;

export default Model.extend({
    title: DS.attr(),
    presenter: DS.attr(),
    time: DS.attr(),
    location: DS.attr(),
    description: DS.attr(),
    formattedTime: computed('time', function() {
        return moment(this.time).format("ddd @ h:mm a, MMMM D, YYYY")
    })
});
```

## Get data in route js

Generate our route to load and display our data: `ember g route events`

In the file `app/routes/events.js` and we will load our data as an API call:

```javascript
import Route from '@ember/routing/route';
import { inject as service } from '@ember/service';

export default Route.extend({
    store: service(),

    model() {
        const store = this.get('store');
        return store.findAll('event');
    }
});
```

Spin up server and let's see what we got
- Run command line `ember serve`
- Navigate to http://localhost:4200

# COMMIT 5

Made events route. Need CORS headers

## Deal With CORS

A Django App that adds CORS (Cross-Origin Resource Sharing) headers to responses. Just what we need! Install with pip into your django project:

`pipenv install django-cors-headers`

Add it to your installed_apps in `settings.py`

```python
INSTALLED_APPS = [
  'django.contrib.admin',
  'django.contrib.auth',
  'django.contrib.contenttypes',
  'django.contrib.sessions',
  'django.contrib.messages',
  'django.contrib.staticfiles',
  'rest_framework',
  'events',
  # corsheaders allows for cross origin communication
  'corsheaders',
]
```

Add it to the top of your middleware:

```python
MIDDLEWARE = [
  # put CorsMiddleware at the top of middleware
  'corsheaders.middleware.CorsMiddleware',
  'django.middleware.security.SecurityMiddleware',
  'django.contrib.sessions.middleware.SessionMiddleware',
```

And lastly, set the mode to debug so no origin shall be denied (**note**: you would not want to put this app on the web this way, but for local development this is fine)

```python
# opens access localhost:8000, including from frontend localhost:4200
CORS_ORIGIN_ALLOW_ALL = DEBUG
```

You may need to stop and start your backend server again, but with a referesh of the frontend, this error should go away. Now how to get the data to display in the browser? We'll work on that next!

## Load Data into the Handlebars

Navigate into `app/templates/events.hbs` and load our model fields and do some styling along with it:

```html
<div class="container">
  <h2 class="display-1">Upcoming Events</h2>
  {{#each model as |event|}}
  <div class="card mb-3">
    <div class="card-header p-3 lead">{{event.title}}</div>
    <ul class="list-group list-group-flush list-group-item-action">
      <li class="list-group-item p-1 pl-3">Who: {{event.presenter}} </li>
      <li class="list-group-item p-1 pl-3">What: {{event.description}}</li>
      <li class="list-group-item p-1 pl-3">Where: {{event.location}}</li>
      <li class="list-group-item p-1 pl-3">When: {{event.formattedTime}}</li>
    </ul>
  </div>
  {{/each}}
</div>
```

Spin up the server and check it out!

# COMMIT 6

Load in events thru route+hbs to browser

## Low Hanging Fruit: Delete

Plan of action:
1. Create an action in `events.hbs` to trigger event in js
2. Create a controller for events.js to handle our action
3. Have action call `destroyRecord` method on the event to delete the instance

### Create an action in events.hbs
Ember action invoked by: `{{action 'delete' event}}`. 
- `action` invokes an action that should be handled in the route or controller
- `'delete'` will be the name of our action in our controller
- `event` is a parameter that will be passed into our action

```html
...truncated...
    <div class="card-header p-3 lead">
      {{event.title}}
      <div class="btn btn-danger btn-lg float-right" role="button" {{action 'delete' event}}>Delete Event</div>
    </div>
    <ul class="list-group list-group-flush list-group-item-action">
...truncated...
```

### Create a crontroller for events.js to handle our action
Using Ember CLI:

`ember g controller events`

Then navigate into the file `app/controllers/events.js`. We will create this action to handle our delete:

```javascript
import Controller from '@ember/controller';

export default Controller.extend({
  actions: {
    delete(event) {
      event.destroyRecord()
    },
  }
});
```

# COMMIT 7

Add delete event action

### 🙌 🙌 Now we have R(etrieve) and D(elete) of CRUD for our REST API Frontend Client

Unfortunately we won't have time to complete all of our CRUD actions in this demonstration. This includes fun things like creating components, passing data down and bubbling actions up. 

Can it get any cooler? 😎 Check out the full tutorial + resource list at: 
## github.com/micleners/DSM-Web-Geeks

## Resources
### Python/Envs:
[Using Pyenv + Pipenv](https://hackernoon.com/reaching-python-development-nirvana-bb5692adf30c).

### Django:
[Django Tutorial](https://docs.djangoproject.com/en/2.2/intro/tutorial01/) << official tutorial straight from the choice

[Excellent Tutorial to Understand Django Rest Framework](https://www.django-rest-framework.org/tutorial/1-serialization/) << starts by showing nuts and bolts then helps you abstract away lines of code in views and serializers via DRF helpers 

### Django + Ember:
[5-part Tutorial on getting Ember and Django Working Togehter 👌](https://medium.com/@sunskyearthwind/django-emberjs-full-stack-basics-connecting-frontend-and-backend-part-1-beed8c386b08) << great tutorial to replicate the process from in this talk

### Ember:
[Ember Tutorial Straight from the Source](https://guides.emberjs.com/release/tutorial/ember-cli/) << includes the latest < AngleBracket /> component invocation

[Excellent Intermediate Ember Tutorial](https://yoember.com/#lesson-1) << build a full CRUD app using Firebase ❤️

[Django Cors Headers](https://github.com/ottoyiu/django-cors-headers) << Cross origin solution for django

### Ember Helpers:
[Ember Bootstrap Addon](https://www.ember-bootstrap.com/) and [Bootstrap](https://getbootstrap.com/) << Gotta love a good CSS framework

[Ember Moment Addon](https://github.com/stefanpenner/ember-moment) and [Moment.js] << time sucks

### Related by Mike:
[My Pyowa Tutorial/Demo on Django + Django Rest Framework in Depth](https://github.com/micleners/pyowa-drf-demo) << if you want to know more about serialization and view helpers in DRF

# ✌️ That's all folks!
<br>

<span style="font-size: 20pt; margin-top: 50px;">Check out the full tutorial + resource list at:</span>
<h1 style="margin: 0; margin-bottom: 50px; font-size: 30pt">github.com/micleners/DSM-Web-Geeks</h1>


<span style="font-size: 20pt">Contact me (Mike Johnson):</span>
<h1 style="margin: 0; font-size: 25pt">mb.johnsonbece@gmail.com</h1>
<h1 style="margin: 0; font-size: 25pt">@mdevstix</h1>

<br>
<h1 style="margin: 0; font-size: 50pt">Questions?</h1>